In [2]:
import pandas as pd
from pathlib import *
import os

In [20]:
def get_docket_data():
    base_path = Path.cwd()
    data_path = Path(Path.cwd().parent.parent, "ArrestCaseDataPipeline", "DCSC-Scraper")
    docket_parsed_files = sorted(data_path.rglob("CF*_docket*"))
    docket_data = pd.concat(pd.read_csv(f) for f in docket_parsed_files) 
    
    return docket_data




In [21]:
data = get_docket_data()

In [27]:
data.head()

Date                                        Docket_Text Image_Avail  \
0  01/01/2007  Event Scheduled Event: Presentment Date: 01/01...         NaN   
1  01/01/2007  Charge Filed Charge #1: Distribution Of a Cont...       Image   
2  01/01/2007  Attorney Appointed/ Attorney DICKEY, MARTHA re...       Image   
3  01/01/2007  Event Scheduled Event: Preliminary Hearing Dat...         NaN   
4  01/01/2007  Event Resulted - Release Status: pr The follow...         NaN   

           Case_ID  Docket_Index  
0  2007 CF2 000003             0  
1  2007 CF2 000003             1  
2  2007 CF2 000003             2  
3  2007 CF2 000003             3  
4  2007 CF2 000003             4

In [58]:
data.dtypes


Date            object
Docket_Text     object
Image_Avail     object
Case_ID         object
Docket_Index     int64
dtype: object

In [59]:
data['Docket_Text'] = data['Docket_Text'].astype(str)

In [60]:
# get each charge - Look out for the variants of Charge #1

def extract_charge(docket):
    if "Charge #1:" in docket:
        return docket.split(":")[1]
    else:
        pass
    
    
data['Charge'] = data['Docket_Text'].apply(extract_charge)

In [64]:
print(data['Charge'].value_counts())

 Distribution Of a Controlled Substance                                                                                                                      6538
 Poss W/I to Dist a Controlled Substance                                                                                                                     5289
 Robbery                                                                                                                                                     2687
 Carry Pistol W/O Lic -Outside Home/Business                                                                                                                 2617
 Assault With A Dangerous Weapon                                                                                                                             2534
 Burglary Two                                                                                                                                                1388
 Bail Reform Act -Felony    

In [ ]:
# Get out case ids with preventitive detention

In [105]:
def aggreate_events():
    data_path = Path(Path.cwd().parent.parent, "ArrestCaseDataPipeline", "DCSC-Scraper")
    events_parsed_files = sorted(data_path.rglob("CF*_event*"))
    events_data = pd.concat(pd.read_csv(f) for f in events_parsed_files) 
    return events_data


In [106]:
events = aggreate_events()

In [107]:
events.shape

(312758, 7)

In [73]:
# Include detention status as a column
detention = ['Preventive Detention', 'Defendant detained pending sentencing','Defendant Held in Other Jurisdiction']
events['detention'] = events['Result'].apply(lambda x: 1 if x in detention else 0 )

In [76]:
events.head()

Date_Time        Location                 Type  \
0  01/01/2007 11:00 AM  Courtroom C-10          Presentment   
1  01/18/2007 09:30 AM   Courtroom 221   Felony Arraignment   
2  01/18/2007 11:00 AM   Courtroom 201  Preliminary Hearing   
3  02/01/2007 09:30 AM   Courtroom 221       Status Hearing   
4  03/02/2007 09:30 AM   Courtroom 221       Status Hearing   

                                             Result Event_Judge  \
0                                      Hearing Held         NaN   
1  Defn Arraigned Plead Not Guilty Jury Demand Made         NaN   
2                                           Vacated         NaN   
3     Status, Preliminary or Other Hearing Not Held         NaN   
4                                      Hearing Held         NaN   

           Case_ID  Event_Index  detention  
0  2007 CF2 000003            0          0  
1  2007 CF2 000003            1          0  
2  2007 CF2 000003            2          0  
3  2007 CF2 000003            3          0  
4  2007 CF2 000003            4          0

In [93]:
preventive_detention = events.loc[events['detention'] == 1]

In [94]:
preventive_detention.shape

(9959, 8)

In [96]:
case_ids = preventive_detention[['Case_ID']]

In [104]:
#case_ids.to_csv('Preventitive_Detention_CaseIDs')